<a href="https://colab.research.google.com/github/DavideEva/Moviefy/blob/main/Moviefy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem analysis

# Import

In [ ]:
# ! pip install tensorflow-addons

In [ ]:
# from tensorflow_addons.layers import InstanceNormalization

In [ ]:
from tensorflow.keras.layers import Layer, InputSpec, LeakyReLU, Input, Conv2D, Activation, Concatenate, Conv2DTranspose, BatchNormalization
from tensorflow import pad
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant

In [ ]:
class ReflectionPadding2D(Layer):
  def __init__(self, padding=(1, 1), **kwargs):
    self.padding = tuple(padding)
    # self.input_spec = [InputSpec(ndim=4)]
    super(ReflectionPadding2D, self).__init__(**kwargs)

  def compute_output_shape(self, s):
    if s[1] == None:
      return (None, None, None, s[3])
    return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

  def call(self, x, mask=None):
    w_pad, h_pad = self.padding
    return tf.pad(x, [[0, 0], [h_pad, h_pad], [w_pad, w_pad], [0, 0]], 'REFLECT')

  def get_config(self):
    config = super(ReflectionPadding2D, self).get_config()
    return config

class Conv2DReflection3x3(Layer):
  def __init__(self, features, stride=1):
    super().__init__()
    self.reflectionPadding2D = ReflectionPadding2D()
    self.conv2d = Conv2D(features, (3,3), strides=(stride, stride), padding='valid', use_bias=False)

  def call(self, inputs, training=False):
    x = self.reflectionPadding2D(inputs, training=training)
    return self.conv2d(x, training=training)


# def Conv2D3x3Reflection(features, stride=1):
#   return Sequential([
#       ReflectionPadding2D(),
#       Conv2D(features, (3,3), strides=(stride, stride), padding='valid', use_bias=False)
#   ], name='Conv2DReflection')


In [ ]:
# define the discriminator model
def define_discriminator(image_shape, alpha=0.2):
  # source image input
  in_image = Input(shape=image_shape)

  # k3n32s1
  d = Conv2DReflection3x3(32, stride=1)(in_image)
  d = LeakyReLU(alpha=alpha)(d)

  # k3n64s2
  d = Conv2DReflection3x3(64, stride=2)(d)
  d = LeakyReLU(alpha=alpha)(d)
  # k3n128s1
  d = Conv2DReflection3x3(128, stride=1)(d)
  d = BatchNormalization(epsilon=1e-5, momentum=0.1, beta_initializer=Constant(1.), gamma_initializer=Constant(0.))(d)
  d = LeakyReLU(alpha=alpha)(d)

  # k3n128s2
  d = Conv2DReflection3x3(128, stride=2)(d)
  d = LeakyReLU(alpha=alpha)(d)
  # k3n256s1
  d = Conv2DReflection3x3(256, stride=1)(d)
  d = BatchNormalization(epsilon=1e-5, momentum=0.1, beta_initializer=Constant(1.), gamma_initializer=Constant(0.))(d)
  d = LeakyReLU(alpha=alpha)(d)

  #feature construction block
  # k3n256s1
  # d = Conv2DReflection3x3(256, stride=1)(d)
  # d = BatchNormalization(epsilon=1e-5, momentum=0.1)(d)
  # d = LeakyReLU(alpha=alpha)(d)

  # patch output
  patch_out = Conv2DReflection3x3(1, stride=1)(d)

  # define model
  model = Model(in_image, patch_out)
  # compile model
  model.compile(loss='mse', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss_weights=[0.5])
  return model

In [ ]:
model = define_discriminator((256,256,3))

In [ ]:
plot_model(model, show_shapes=True, expand_nested=True)